In [25]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import plotly.express as px


# Carregar dados
df_campaign_queue = pd.read_csv("CampaignQueue.csv", sep=";")

# Converter scheduledAt para datetime e extrair hora
df_campaign_queue['scheduledAt'] = pd.to_datetime(df_campaign_queue['scheduledAt'], dayfirst=True, errors='coerce')
df_campaign_queue['hour'] = df_campaign_queue['scheduledAt'].dt.hour

# Calcular taxa de leitura por campanha
taxa_leitura = df_campaign_queue.groupby('campaignId')['status'].apply(
    lambda x: (x == 'Read').sum() / len(x) * 100
).reset_index(name='taxa_leitura')

# Mesclar taxa_leitura com df_campaign_queue
df_agg = df_campaign_queue.merge(taxa_leitura, on='campaignId', how='left')
# Preencher valores nulos sem inplace
df_agg['taxa_leitura'] = df_agg['taxa_leitura'].fillna(0)

# Definir variáveis fuzzy
hour = ctrl.Antecedent(np.arange(0, 24, 1), 'hour')  # Hora do dia (0-23)
taxa_leitura_fuzzy = ctrl.Antecedent(np.arange(0, 101, 1), 'taxa_leitura')  # 0-100%
priority = ctrl.Consequent(np.arange(0, 101, 1), 'priority')  # Prioridade (0-100%)

# Funções de pertinência
hour['manhã'] = fuzz.trimf(hour.universe, [0, 6, 12])
hour['tarde'] = fuzz.trimf(hour.universe, [10, 15, 18])
hour['noite'] = fuzz.trimf(hour.universe, [16, 20, 24])

taxa_leitura_fuzzy['baixa'] = fuzz.trimf(taxa_leitura_fuzzy.universe, [0, 0, 40])
taxa_leitura_fuzzy['média'] = fuzz.trimf(taxa_leitura_fuzzy.universe, [30, 50, 70])
taxa_leitura_fuzzy['alta'] = fuzz.trimf(taxa_leitura_fuzzy.universe, [60, 100, 100])

priority['baixa'] = fuzz.trimf(priority.universe, [0, 0, 40])
priority['média'] = fuzz.trimf(priority.universe, [30, 50, 70])
priority['alta'] = fuzz.trimf(priority.universe, [60, 100, 100])

# Regras fuzzy (expandidas para melhor cobertura)
rule1 = ctrl.Rule(hour['tarde'] & taxa_leitura_fuzzy['alta'], priority['alta'])
rule2 = ctrl.Rule(hour['manhã'] & taxa_leitura_fuzzy['média'], priority['média'])
rule3 = ctrl.Rule(hour['noite'] & taxa_leitura_fuzzy['baixa'], priority['baixa'])
rule4 = ctrl.Rule(hour['manhã'] & taxa_leitura_fuzzy['alta'], priority['média'])
rule5 = ctrl.Rule(hour['tarde'] & taxa_leitura_fuzzy['média'], priority['média'])
rule6 = ctrl.Rule(hour['noite'] & taxa_leitura_fuzzy['alta'], priority['alta'])
rule7 = ctrl.Rule(hour['manhã'] & taxa_leitura_fuzzy['baixa'], priority['baixa'])
rule8 = ctrl.Rule(hour['tarde'] & taxa_leitura_fuzzy['baixa'], priority['média'])
rule9 = ctrl.Rule(hour['noite'] & taxa_leitura_fuzzy['média'], priority['média'])

# Sistema de controle
priority_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
priority_sim = ctrl.ControlSystemSimulation(priority_ctrl)

# Aplicar lógica fuzzy
df_agg['priority'] = 0.0
for idx, row in df_agg.iterrows():
    # Validar entradas dentro dos intervalos
    hour_val = min(max(row['hour'] if pd.notna(row['hour']) else 12, 0), 23)
    taxa_leitura_val = min(max(row['taxa_leitura'], 0), 100)

    priority_sim.input['hour'] = hour_val
    priority_sim.input['taxa_leitura'] = taxa_leitura_val
    try:
        priority_sim.compute()
        if 'priority' in priority_sim.output:
            df_agg.at[idx, 'priority'] = priority_sim.output['priority']
        else:
            print(f"\nLinha {idx}: Saída 'priority' não encontrada\nUsando valor padrão 50\n")
            df_agg.at[idx, 'priority'] = 50
    except Exception as e:
        print(f"\nLinha {idx}: Erro ao calcular - {e}\nUsando valor padrão 50\n")
        df_agg.at[idx, 'priority'] = 50

# Visualizar resultados no dashboard com melhor legibilidade
fig = px.histogram(df_agg, x='priority', color='status',
                   title='Distribuição de Prioridade de Disparo por Status da Campanha',
                   labels={'priority': 'Prioridade de Disparo (%)', 'status': 'Status da Campanha'},
                   barmode='group',  # Barras agrupadas para comparação clara
                   height=700,       # Aumentar altura
                   width=900,        # Aumentar largura
                   nbins=10)         # Limitar número de bins para evitar excesso
fig.update_layout(
    margin=dict(l=60, r=60, t=120, b=60),  # Margens maiores
    title_x=0.5,                           # Centralizar título
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, title_text='Status'),  # Legenda otimizada
    xaxis_title="Prioridade de Disparo (%)",
    yaxis_title="Contagem"
)
fig.update_traces(opacity=0.7)  # Reduzir opacidade para evitar sobreposição
fig.show()

# Exportar sistema fuzzy
import joblib
joblib.dump(priority_sim, 'fuzzy_campaign_priority.pkl')


Linha 38: Saída 'priority' não encontrada
Usando valor padrão 50



['fuzzy_campaign_priority.pkl']

In [6]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.6 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import plotly.express as px

# Carregar dados
df_campaign_queue = pd.read_csv("CampaignQueue.csv", sep=";")
df_orders = pd.read_csv("Order.csv", sep=";")

# Calcular entradas fuzzy
# Número de mensagens por campanha
num_mensagens = df_campaign_queue.groupby('campaignId').size().reset_index(name='num_mensagens')

# Taxa de leitura (% de status "Read")
taxa_leitura = df_campaign_queue.groupby('campaignId')['status'].apply(
    lambda x: (x == 'Read').sum() / len(x) * 100
).reset_index(name='taxa_leitura')

# Valor médio dos pedidos (vinculado via customerId -> customer)
df_merged = pd.merge(df_campaign_queue, df_orders, left_on='customerId', right_on='customer', how='left')
valor_medio = df_merged.groupby('campaignId')['totalAmount'].mean().reset_index(name='valor_medio')

# Combinar entradas
df_agg = num_mensagens.merge(taxa_leitura, on='campaignId').merge(valor_medio, on='campaignId')
df_agg.fillna({'num_mensagens': 0, 'taxa_leitura': 0, 'valor_medio': 0}, inplace=True)

# Definir variáveis fuzzy
num_mensagens_fuzzy = ctrl.Antecedent(np.arange(0, 201, 1), 'num_mensagens')  # 0-200 mensagens
taxa_leitura_fuzzy = ctrl.Antecedent(np.arange(0, 101, 1), 'taxa_leitura')  # 0-100%
valor_medio_fuzzy = ctrl.Antecedent(np.arange(0, 2001, 1), 'valor_medio')  # 0-2000
sucesso = ctrl.Consequent(np.arange(0, 101, 1), 'sucesso')  # Sucesso (0-100%)

# Funções de pertinência
num_mensagens_fuzzy['baixo'] = fuzz.trimf(num_mensagens_fuzzy.universe, [0, 0, 50])
num_mensagens_fuzzy['médio'] = fuzz.trimf(num_mensagens_fuzzy.universe, [30, 75, 120])
num_mensagens_fuzzy['alto'] = fuzz.trimf(num_mensagens_fuzzy.universe, [100, 200, 200])

taxa_leitura_fuzzy['baixa'] = fuzz.trimf(taxa_leitura_fuzzy.universe, [0, 0, 40])
taxa_leitura_fuzzy['média'] = fuzz.trimf(taxa_leitura_fuzzy.universe, [30, 50, 70])
taxa_leitura_fuzzy['alta'] = fuzz.trimf(taxa_leitura_fuzzy.universe, [60, 100, 100])

valor_medio_fuzzy['baixo'] = fuzz.trimf(valor_medio_fuzzy.universe, [0, 0, 400])
valor_medio_fuzzy['médio'] = fuzz.trimf(valor_medio_fuzzy.universe, [300, 600, 900])
valor_medio_fuzzy['alto'] = fuzz.trimf(valor_medio_fuzzy.universe, [800, 2000, 2000])

sucesso['baixo'] = fuzz.trimf(sucesso.universe, [0, 0, 40])
sucesso['médio'] = fuzz.trimf(sucesso.universe, [30, 50, 70])
sucesso['alto'] = fuzz.trimf(sucesso.universe, [60, 100, 100])

# Regras fuzzy
rule1 = ctrl.Rule(num_mensagens_fuzzy['alto'] & taxa_leitura_fuzzy['alta'] & valor_medio_fuzzy['alto'], sucesso['alto'])
rule2 = ctrl.Rule(num_mensagens_fuzzy['médio'] & taxa_leitura_fuzzy['média'] & valor_medio_fuzzy['médio'], sucesso['médio'])
rule3 = ctrl.Rule(num_mensagens_fuzzy['baixo'] | taxa_leitura_fuzzy['baixa'] | valor_medio_fuzzy['baixo'], sucesso['baixo'])
rule4 = ctrl.Rule((num_mensagens_fuzzy['médio'] | taxa_leitura_fuzzy['média']) & valor_medio_fuzzy['alto'], sucesso['médio'])
rule5 = ctrl.Rule(num_mensagens_fuzzy['alto'] & taxa_leitura_fuzzy['média'], sucesso['médio'])
rule6 = ctrl.Rule(num_mensagens_fuzzy['baixo'] & taxa_leitura_fuzzy['alta'] & valor_medio_fuzzy['alto'], sucesso['médio'])
rule7 = ctrl.Rule(num_mensagens_fuzzy['alto'] & taxa_leitura_fuzzy['baixa'], sucesso['baixo'])
rule8 = ctrl.Rule(taxa_leitura_fuzzy['alta'] & valor_medio_fuzzy['médio'], sucesso['médio'])

# Sistema de controle
sucesso_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])
sucesso_sim = ctrl.ControlSystemSimulation(sucesso_ctrl)

# Aplicar lógica fuzzy
df_agg['sucesso'] = 0.0
for idx, row in df_agg.iterrows():
    # Validar entradas
    num_mensagens = min(max(row['num_mensagens'], 0), 200)
    taxa_leitura = min(max(row['taxa_leitura'], 0), 100)
    valor_medio = min(max(row['valor_medio'], 0), 2000) if pd.notna(row['valor_medio']) else 0

    sucesso_sim.input['num_mensagens'] = num_mensagens
    sucesso_sim.input['taxa_leitura'] = taxa_leitura
    sucesso_sim.input['valor_medio'] = valor_medio
    try:
        sucesso_sim.compute()
        if 'sucesso' in sucesso_sim.output:
            df_agg.at[idx, 'sucesso'] = sucesso_sim.output['sucesso']
        else:
            print(f"Erro na linha {idx}: Saída 'sucesso' não encontrada")
            df_agg.at[idx, 'sucesso'] = 50
    except Exception as e:
        print(f"Erro na linha {idx}: {e}")
        df_agg.at[idx, 'sucesso'] = 50  # Valor padrão para erros

# Visualizar resultados no dashboard
fig = px.scatter(df_agg, x='taxa_leitura', y='sucesso', size='num_mensagens', color='valor_medio',
                 title='Sucesso das Campanhas por Taxa de Leitura e Valor Médio dos Pedidos',
                 labels={'taxa_leitura': 'Taxa de Leitura (%)', 'sucesso': 'Grau de Sucesso (%)',
                         'valor_medio': 'Valor Médio dos Pedidos'})
fig.show()

# Testar uma campanha específica
try:
    sucesso_sim.input['num_mensagens'] = 50
    sucesso_sim.input['taxa_leitura'] = 70
    sucesso_sim.input['valor_medio'] = 200
    sucesso_sim.compute()
    if 'sucesso' in sucesso_sim.output:
        print(f"Nível de Sucesso da Campanha: {sucesso_sim.output['sucesso']:.2f}%")
    else:
        print("Erro: Saída 'sucesso' não encontrada. Usando valor padrão.")
        print("Nível de Sucesso da Campanha: 50.00%")
except Exception as e:
    print(f"Erro ao testar campanha: {e}")
    print("Nível de Sucesso da Campanha: 50.00% (padrão)")

# Exportar sistema fuzzy
import joblib
joblib.dump(sucesso_sim, 'fuzzy_campaign_success.pkl')

Nível de Sucesso da Campanha: 15.56%


['fuzzy_campaign_success.pkl']